In [1]:
import os, gspread, pickle, re
from telethon import TelegramClient, sync, events
from telethon.tl.types import Chat, User, PeerUser
from datetime import date
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
def get_id_from_string(str):
    try:
        id = int(str)
        return id
    except:
        pass

def get_admins_list(filename):
    file = open(filename, 'r')
    content = file.read()
    list_with_None = list(map(get_id_from_string, content.split('\n')))
    final_list = [x for x in list_with_None if x]
    return final_list

def get_whitelist_ids(filename):
    file = open(filename, 'r')
    content = file.read()
    list_with_None = [get_id_from_string(x.split(' ')[0]) for x in content.split('\n')]
    final_list = [x for x in list_with_None if x]
    return final_list

def get_spreadsheets_by_id(filename, user_id):
    file = open(filename, 'r')
    content = file.read()
    list_with_None = [x.split(' ')[1] for x in content.split('\n') if get_id_from_string(x.split(' ')[0]) == user_id]
    final_list = [x for x in list_with_None if x]
    return final_list

def get_spreadsheets_list(filename):
    file = open(filename, 'r')
    content = file.read()
    list_with_None = content.split('\n')
    final_list = [x for x in list_with_None if x]
    return final_list

def get_credentials(file_name):
    file = open('auth/{}'.format(file_name), 'r')
    content = file.read()
    credentials_with_None = content.split('\n')
    credentials = [x for x in credentials_with_None if x]
    return {'api_id': int(credentials[0]), 'api_hash': credentials[1], 'bot_token': credentials[2]}

In [3]:
# Use your own values here
creds = get_credentials('credentials.txt')

telegram_client = TelegramClient('bot_session', creds['api_id'], creds['api_hash'])
telegram_client.start(bot_token=creds['bot_token'])

In [4]:
date_col, proxyIP_col, proxyLogin_col, proxyPass_col = 4, 8, 10, 12

@telegram_client.on(events.NewMessage())
def handler(event):
    if isinstance(event.message.to_id, PeerUser) and event.message.from_id != telegram_client.get_me().id:
        is_admin = event.message.from_id in get_admins_list('data/admins.txt')
        is_whitelist_user = event.message.from_id in get_whitelist_ids('data/whitelist.txt')
        
        if is_whitelist_user and event.message.message in [x + '_report' for x in get_spreadsheets_list('data/spreadsheets.txt')]:
            for user_id in get_admins_list('data/admins.txt'):
                telegram_client.send_message(user_id, 'Этот пользователь запросил доступ к выгрузке статистики для учётных записей из таблицы')
                telegram_client.forward_messages(user_id, event.message)
                telegram_client.send_message(user_id, f'Чтобы дать ему доступ просто внесите в файл whitelist.txt на сервере следующую строку:\n\n```{event.message.from_id} {event.message.message[:-7]}```')
            telegram_client.send_message(event.message.from_id, 'Ваш запрос отправлен администраторам')
        elif is_admin or is_whitelist_user:
            try:
                login = event.message.message[:event.message.message.find(' ')].strip()
                campaign = event.message.message[event.message.message.find(' '):].strip()
                campaign_number = campaign.split('№')[1][1:-1]
                session_filename = f'{login.split("@")[0] + "@" + login.split("@")[1].split(".")[0]}.pkl'
                session = None
                target_spreadsheet_name = None
                source_spreadsheet_name = None
                
                spreadsheet_list = get_spreadsheets_list('data/spreadsheets.txt') if is_admin else get_spreadsheets_by_id('data/whitelist.txt', event.message.from_id)
                for spreadsheet_name in spreadsheet_list:
                    if os.path.isfile(f'./yandex_sessions/{spreadsheet_name}_report/{session_filename}'):
                        with open(f'./yandex_sessions/{spreadsheet_name}_report/{session_filename}', 'rb') as f:
                            session = pickle.load(f)
                            source_spreadsheet_name = spreadsheet_name
                            target_spreadsheet_name = spreadsheet_name + '_report'
                if session == None:
                    raise FileNotFoundError
                
                scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
                creds = ServiceAccountCredentials.from_json_keyfile_name('auth/client_secret.json', scope)
                google_client = gspread.authorize(creds)
                
                source_spreadsheet = google_client.open(source_spreadsheet_name).sheet1
                source_row = source_spreadsheet.find(login).row
                target_spreadsheet = google_client.open(target_spreadsheet_name).sheet1
                target_row = target_spreadsheet.find(f'{campaign.split("№")[0]}№\xa0{campaign_number})').row
                
                date1 = target_spreadsheet.cell(target_row, date_col).value
                [d1, m1, y1] = date1.split('.')
                date2 = date.today().strftime('%d.%m.%y')
                [d2, m2, y2] = date2.split('.')
                
                proxyIP = source_spreadsheet.cell(source_row, proxyIP_col).value
                proxyLogin = source_spreadsheet.cell(source_row, proxyLogin_col).value
                proxyPass = source_spreadsheet.cell(source_row, proxyPass_col).value
                
                proxies = {
                    'http': f'http://{proxyLogin}:{proxyPass}@{proxyIP}',
                    'https': f'https://{proxyLogin}:{proxyPass}@{proxyIP}',
                }
                
                xls_url = f'https://direct.yandex.ru/registered/{date1}-{date2}_main{campaign_number}.xls?cmd=showCampStat&y1={y1}&types=total&ulogin={login.split("@")[0]}&with_nds&y2={y2}&isStat=1&m1={m1}&m2={m2}&d2={d2}&d1={d1}&cid={campaign_number}&sort=text&reverse&xls=1'
                xls_file = session.get(xls_url, proxies=proxies)
                with open(f'./{login} {campaign}.xls', 'wb') as f:
                    f.write(xls_file.content)
                with open(f'./{login} {campaign}.xls', 'rb') as f:
                    telegram_client.send_file(event.message.from_id, f)
                os.remove(f'./{login} {campaign}.xls')
            except FileNotFoundError:
                telegram_client.send_message(event.message.from_id, 'Такой учётной записи нет')
            except gspread.CellNotFound:
                telegram_client.send_message(event.message.from_id, 'Такой кампании нет')
            except Exception:
                telegram_client.send_message(event.message.from_id, 'Вы что-то неправильно ввели.\n\nЕсли хотите выгрузить файл статистики — отправьте логин и название кампании через пробел, например:\n\n```yandex@yandex.ru Новая (№ 12345678)```')
                if is_whitelist_user:
                    telegram_client.send_message(event.message.from_id, 'Если хотите запросить доступ к ещё одной таблице таблице — отправьте её название.')
        else:
            telegram_client.send_message(event.message.from_id, 'Такой таблицы нет\n\n Необходимо отправить правильное название таблицы, чтобы запросить доступ к выгрузке статистики')